In [ ]:
using MIToS
using MIToS.MSA
using MIToS.Information
using StatsBase
using Plots
using LinearAlgebra
using Distances
using Clustering
using PairwiseListMatrices
using Statistics
using GraphRecipes
using DataFrames
using StatsPlots 

In [ ]:
# Truncate IJulia outputs at:
ENV["LINES"]   = 20 
ENV["COLUMNS"] = 600;

## Reading MSA files
### The main function for reading MSA files in MIToS is read and it is defined in the Utils module. This function takes a filename/path as a first argument followed by other arguments. It opens the file and uses the arguments to call the parse function. read decides how to open the file, using the prefixes (e.g. https) and suffixes (i.e. extensions) of the file name, while parse does the actual parsing of the file. You can read gzipped files if they have the .gz extension and also urls pointing to a web file. The second argument of read and parse is the file FileFormat. The supported MSA formats at the moment are Stockholm, FASTA, PIR (NBRF) and Raw. 

In [ ]:
fasta_file = "top2a.fa"
#fasta_file = ""
msa = read(fasta_file, FASTA, generatemapping=true, useidcoordinates=true)
println("This MSA has ", nsequences(msa), " sequences...")
#fig = plotmsa(msa; colorscheme = :tableau_blue_green)

## Limiting the data from 556 to 1000 sequences

In [ ]:
msa = msa[:,556:1000]

## Describing your MSA
### The MSA module has a number of functions to gain insight about your MSA. 
### Using MIToS.MSA, one can easily ask for:

#### The number of columns and sequences with the ncolumns and nsequences functions.
#### The fraction of columns with residues (coverage) for each sequence making use of the coverage method.

#### The fraction or percentage of gaps/residues using with the functions gapfraction, residuefraction and columngapfraction.

#### The percentage of identity (PID) between each sequence of the MSA or its mean value

In [ ]:
coverage(msa)

#### The gapfraction and coverage functions return a vector of numbers between 0.0 and 1.0 (fraction of...). Sometime it's useful to plot this data to quickly understand the MSA structure. In this example, we are going to use the Plots package for plotting, with the GR backend, but you are free to use any of the Julia plotting libraries.

In [ ]:
columngapfraction(msa)

In [ ]:
gr(size=(600,300))

plot(   1:ncolumns(msa), # x is a range from 1 to the number of columns
        vec(columngapfraction(msa)) .* 100.0, # y is a Vector{Float64} with the percentage of gaps of each column
        linetype = :line,
        ylabel = "gaps [%]",
        xlabel = "columns",
        legend=false)

In [ ]:
plot(   1:nsequences(msa), # x is a range from 1 to the number of sequences
        vec(coverage(msa)) .* 100, # y is a Vector{Float64} with the coverage of each sequence
        linetype = :line,
        ylabel = "coverage [%]",
        xlabel = "sequences",
        legend=false)

In [ ]:
plot(msa)

#### Example: Filter sequences per coverage and columns per gap fraction
#### Taking advantage of the filter...! functions and the coverage and columngapfraction functions, it's possible to delete short sequences or columns with a lot of gaps.

In [ ]:
println("\tsequences\tcolumns")
println( "Before:\t", nsequences(msa), "\t\t", ncolumns(msa)  )
# delete sequences with less than 90% coverage of the MSA length:
filtersequences!(msa, coverage(msa) .>= 0.9)
# delete columns with more than 10% of gaps:
filtercolumns!(msa, columngapfraction(msa) .<= 0.1)
println( "After:\t", nsequences(msa), "\t\t",  ncolumns(msa)  )

In [ ]:
histogram(  vec(columngapfraction(msa)),
            # Using vec() to get a Vector{Float64} with the fraction of gaps of each column
            xlabel = "gap fraction in [0,1]", bins = 10, legend = false)

In [ ]:
histogram(  vec(coverage(msa) .* 100.0), #  Column with the coverage of each sequence
            xlabel = "coverage [%]", legend=false)

### Plotting the percentage of identity between sequences
#### The distribution of the percentage of identity between every pair of sequences in an MSA, gives an idea of the MSA diversity. In this example, we are using percentidentity over an MSA to get those identity values.

In [ ]:
pid = percentidentity(msa)

In [ ]:
pidtable = to_table(pid, diagonal=false)

In [ ]:
quantile(convert(Vector{Float64}, pidtable[:,3]), [0.00, 0.25, 0.50, 0.75, 1.00])

In [ ]:
meanpercentidentity(msa)

In [ ]:
gr()
heatmap(convert(Matrix, pid), yflip=true, ratio=:equal)

In [ ]:
histogram(pidtable[:,3], xlabel ="Percentage of identity", legend=false)

In [ ]:
getresidues(msa)

In [ ]:
getresiduesequences(msa)

In [ ]:
sequencenames(msa)

## MIToS.MSA.gapstrip

Creates a new matrix of Residues (MSA) with deleted sequences and columns/positions. The MSA is edited in the following way:

Removes all the columns/position on the MSA with gaps on the reference (first) sequence
Removes all the sequences with a coverage with respect to the number of
columns/positions on the MSA less than a coveragelimit (default to 0.75: sequences with 25% of gaps)

Removes all the columns/position on the MSA with more than a gaplimit
(default to 0.5: 50% of gaps)

In [ ]:
msa1 = MIToS.MSA.gapstrip(msa,gaplimit=0.65)

### Because matrices are stored columnwise in Julia, we use getresiduesequences function when we need to heavily operate over sequences.

In [ ]:
residues = getresidues(msa1) # estimateincolumns functions take a Matrix{Residue}

## Count residues to estimate the entropy. 
The entropy estimation is performed over a rehused Counts object. The result will be a vector containing the values estimated over each column without counting gaps (UngappedAlphabet).

In [ ]:

Hx = mapcolfreq!(entropy, msa, Counts(ContingencyTable(Float64, Val{1}, UngappedAlphabet())))

#### In the above examples, we indicate the type of each occurrence in the counting and the probability table to use. Also, it's possible for some measures as entropy and mutual information, to estimate the values only with the count table (without calculate the probability table). 
#### Estimating measures only with a ResidueCount table, when this is possible, should be faster than using a probability table.

In [ ]:
#Hxy = mapcolpairfreq!(entropy, msa1, Counts(ContingencyTable(Float64, Val{2}, UngappedAlphabet())))

In [ ]:
Time_Pab = map(1:100) do x
    time = @elapsed mapcolpairfreq!(entropy, msa, Probabilities(ContingencyTable(Float64, Val{2}, UngappedAlphabet())))
end

Time_Nab = map(1:100) do x
    time = @elapsed mapcolpairfreq!(entropy, msa, Counts(ContingencyTable(Float64, Val{2}, UngappedAlphabet())))
end

using Plots
gr()

histogram( [Time_Pab Time_Nab],
    labels = ["Using ResidueProbability" "Using ResidueCount"],
    xlabel = "Execution time [seconds]" )

### Estimating H(X) and H(X, Y) over an MSA
#### In this example, we are going to use mapcolfreq! and mapcolpairfreq! to estimate Shannon entropy of MSA columns H(X) and the joint entropy H(X, Y) of columns pairs, respectively.

In [ ]:
NMIxy = mapcolpairfreq!(normalized_mutual_information, msa1, Counts(ContingencyTable(Float64, Val{2}, GappedAlphabet())), Val{false})

In [ ]:
NMIxy_transpose = mapcolpairfreq!(normalized_mutual_information, transpose(msa1), Counts(ContingencyTable(Float64, Val{2}, GappedAlphabet())), Val{false})

In [ ]:
NMI_matrix_t = convert(Matrix{Float64}, NMIxy_transpose.array)

## K-means
### K-means is a classical method for clustering or vector quantization. It produces a fixed number of clusters, each associated with a center (also known as a prototype), and each data point is assigned to a cluster with the nearest center.

### From a mathematical standpoint, K-means is a coordinate descent algorithm that solves the following optimization problem:

\begin{equation}
			\text{min.}_{C_1,\ldots,C_K} \sum_{k=1}^{k} \frac{1}{|C_k|} \sum_{i, i^{'} \in C_k} \sum_{j=1}^{p} (x_{ij} - x_{i^{'}j})^2
\end{equation}


### Number of Clusters = 5

In [ ]:
num_cluster = 5

In [ ]:
# cluster X into 20 clusters using K-means 
KClusters = kmeans(NMI_matrix_t, num_cluster; maxiter = 100,  
                                display=:iter)

In [ ]:
# verify the number of clusters 
nclusters(KClusters) == 5

In [ ]:
# get the assignments of points to clusters 
cluster_assignment = assignments(KClusters) 

In [ ]:
# get the cluster sizes 
cnt = counts(KClusters) 

In [ ]:
# get the cluster centers 
Cluster_Center = KClusters.centers 

In [ ]:
df = DataFrame( seqnum = 1:nsequences(msa1),
                seqname = sequencenames(msa1),
                cluster = cluster_assignment, # the cluster number/index of each sequence
                coverage = vec(coverage(msa1)))

first(df, 20)

### Group by clusters and calculate summary statistics

In [ ]:

top_clusters = combine(
    groupby(df, :cluster),
    :seqnum => length => :n_sequences,
    :coverage => mean => :mean_coverage
)


### Display the top clusters


first(top_clusters, 20)  # Display the top 10 clusters

In [ ]:
# Assuming you already have the df DataFrame with the specified columns
df_filtered = filter(row -> row.cluster > 1, df)

first(df_filtered, 20)

### Plot using Histogram

In [ ]:
h = @df df histogram(:cluster, ylabel="nseq")
p = @df df plot(:cluster, :coverage, linetype=:scatter)
plot(p, h, nc=1, xlim=(0, nclusters(KClusters)+1 ), legend=false)

In [ ]:
maxcoverage = by(df, :cluster, cl -> cl[ findmax(cl[:coverage])[2] ,
                 [:seqnum, :seqname, :coverage]])

first(maxcoverage, 20)

### We use the Split-Apply-Combine strategy, though the by function of the DataFrames package, to select the sequence of highest coverage for each cluster.

In [ ]:
p = @df maxcoverage plot(:cluster, :coverage, linetype=:scatter)
h = @df maxcoverage histogram(:cluster, ylabel="nseq")
plot(p, h, nc=1, xlim=(0, nclusters(KClusters)+1 ), legend=false)
png("msa_clusters_iii.png") # hide
nothing # hide

### We can easily generate a mask using list comprehension, to select only the representative sequences of the MSA (deleting the rest of the sequences with filtersequences!).

In [ ]:
cluster_references = Bool[ seqnum in maxcoverage[:seqnum] for seqnum in 1:nsequences(msa1) ]

In [ ]:
filtersequences!(msa1, cluster_references)

In [ ]:
plot(msa1)

In [ ]:
using Plots

sum(KClusters.assignments.==5)

In [ ]:
using GraphRecipes

graphplot(NMI_matrix_t[1:10,1:10])

In [ ]:
graphplot(UpperTriangular(NMI_matrix_t[1:10,1:10]))

In [ ]:
scatter(KClusters.assignments)

In [ ]:
heatmap(log2.(NMI_matrix_t), yflip=true)

In [ ]:
# Create a scatter plot
scatter(1:size(NMI_matrix_t, 2), NMI_matrix_t, color=cluster_assignment, legend=false)
xlabel!("position")
ylabel!("seq")



### Principal Component Analysis (PCA) Plot
Since the data has a high dimension, we can use PCA to reduce it to a few principal components and then plot the clusters.

In [ ]:
using MultivariateStats

# Perform PCA
pca_result = fit(PCA, NMI_matrix_t', maxoutdim=2)
pca_data = MultivariateStats.transform(pca_result, NMI_matrix_t')

# Create a PCA plot
scatter(pca_data[1, :], pca_data[2, :], color=cluster_assignment, legend=false)
xlabel!("Principal Component 1")
ylabel!("Principal Component 2")

In [ ]:
scatter3d(NMI_matrix_t[1, :], NMI_matrix_t[2, :], NMI_matrix_t[3, :], color=cluster_assignment, legend=false)
